<a href="https://colab.research.google.com/github/averma111/Pytorch-Learning/blob/main/feedforwardneuralnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper Parameters
input_size = 784
hidden_size = 500
num_classes = 10
epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [7]:
# Fully connected neural network with hidden layers
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
  def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [8]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [9]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [10]:
# Train the model
total_step = len(train_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.5188
Epoch [1/5], Step [200/600], Loss: 0.2421
Epoch [1/5], Step [300/600], Loss: 0.3487
Epoch [1/5], Step [400/600], Loss: 0.1951
Epoch [1/5], Step [500/600], Loss: 0.3033
Epoch [1/5], Step [600/600], Loss: 0.1463
Epoch [2/5], Step [100/600], Loss: 0.1366
Epoch [2/5], Step [200/600], Loss: 0.0665
Epoch [2/5], Step [300/600], Loss: 0.2024
Epoch [2/5], Step [400/600], Loss: 0.0832
Epoch [2/5], Step [500/600], Loss: 0.0762
Epoch [2/5], Step [600/600], Loss: 0.1083
Epoch [3/5], Step [100/600], Loss: 0.1405
Epoch [3/5], Step [200/600], Loss: 0.0944
Epoch [3/5], Step [300/600], Loss: 0.0394
Epoch [3/5], Step [400/600], Loss: 0.0589
Epoch [3/5], Step [500/600], Loss: 0.1341
Epoch [3/5], Step [600/600], Loss: 0.0414
Epoch [4/5], Step [100/600], Loss: 0.0890
Epoch [4/5], Step [200/600], Loss: 0.0705
Epoch [4/5], Step [300/600], Loss: 0.0654
Epoch [4/5], Step [400/600], Loss: 0.0090
Epoch [4/5], Step [500/600], Loss: 0.0365
Epoch [4/5], Step [600/600], Loss:

In [11]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.15 %


In [12]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')